# Reading and writing Nastran output4 files

This and other notebooks are available here: ``kittyhawk:/home/loads/twidrick/code/pyyeti/docs/tutorials``.

First, do some imports:

In [ ]:
import numpy as np
import os
from pyyeti import op4

Generate a couple matrices to write to disk:

In [ ]:
a = np.random.randn(4, 10)
b = np.random.randn(5, 14)

---
#### Basic writing
Write them to a file in a random order. By default, the file with be a native-endian binary and the matrices will be written in dense (non-sparse) format.

In [ ]:
filename = 'rw_op4_demo.op4'
op4.write(filename, dict(a=a, b=b))

---
#### Basic reading
Read them back in and compare. The following reads the matrices into a dictionary:

In [ ]:
dct = op4.read(filename)
assert np.all(dct['a'] == a)
assert np.all(dct['b'] == b)

The "pretty-print" class from pyYeti can help in viewing the dictionary:

In [ ]:
from pyyeti.pp import PP
PP(dct);

---
If you need the form and type, set the `justmatrix` option to False (or use the `load` function ... it is the same as `read` except the default on `justmatrix`). In this case, dictionary entry is a 3-element tuple: (matrix, form, type):

In [ ]:
dct = op4.read(filename, justmatrix=False)
assert np.all(dct['a'][0] == a)
assert np.all(dct['b'][0] == b)
PP(dct);

---
#### Ascii save

As noted above, matrices are written in binary by default. To write in ASCII format, set `binary` to False:

In [ ]:
op4.write(filename, dict(a=a, b=b), binary=False)

In [ ]:
with open(filename) as f:
    for i in range(5):
        print(f.readline().strip())

---
#### Writing in a certain order
To ensure that "`b`" gets saved before "`a`" you have to use the list inputs.

In [ ]:
op4.write(filename, ['b', 'a'], [b, a], binary=False)
with open(filename) as f:
    for i in range(5):
        print(f.readline().strip())

---
#### Reading into list
If the output4 file has duplicate names, you'll have to read the variables into a list:

In [ ]:
b2 = b + 10
op4.write(filename, ['b', 'a', 'b'], [b, a, b2])
names, mats, *_ = op4.read(filename, into='list')
names

In [ ]:
assert np.all(b == mats[0])
assert np.all(a == mats[1])
assert np.all(b2 == mats[2])

Clean up:

In [ ]:
os.remove(filename)